# Initialization Step

In [1]:
# Import the provided python time-series API
from qids_package.qids import *

# Initialize the environment
env = make_env()

# Import other packages you will need
!pip install talib-binary
import copy
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
import datetime
import talib as ta
from sklearn.model_selection import TimeSeriesSplit

# File address for train datasets
TRAIN_MARKET_PATH = "/kaggle/input/hku-qids-2023-quantitative-investment-competition/first_round_train_market_data.csv"
TRAIN_FUNADMENTAL_PATH = "/kaggle/input/hku-qids-2023-quantitative-investment-competition/first_round_train_fundamental_data.csv"
TRAIN_RETURN_PATH = "/kaggle/input/hku-qids-2023-quantitative-investment-competition/first_round_train_return_data.csv"

Environment is initialized.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.8 MB/s eta 0:00:00


# My Feature Library

In [2]:
# Feature构建函数：过去1天，5天，10天，20天的close-to-close收益率。同时也添加最新的价格作为feature。
# 这里输入的数据为sAdBpC格式
def construct_ctc_returns(market_data_df):
    
    # 选出正确时间点的数据
    selected_data_df = market_data_df[market_data_df.index.map(lambda x: True if x.split("p")[1] == "50" else False)]
    selected_data_df.index = selected_data_df.index.map(lambda x: x.split("p")[0])
    
    # 记录计算结果
    result_df = pd.DataFrame()
    
    # Add some naive features
    result_df["open_price"] = selected_data_df.open
    result_df["close_price"] = selected_data_df.close
    result_df["high_price"] = selected_data_df.high
    result_df["low_price"] = selected_data_df.low
    result_df["volume"] = selected_data_df.volume
    result_df["money"] = selected_data_df.money
    
    # Add the target features
    result_df["ctc_return_1d"] = selected_data_df.close.pct_change(1)
    result_df["ctc_return_5d"] = selected_data_df.close.pct_change(5)
    result_df["ctc_return_10d"] = selected_data_df.close.pct_change(10)
    result_df["ctc_return_20d"] = selected_data_df.close.pct_change(20)

    # 返回结果
    return result_df

# Feature构建函数：过去1天的交易量和过去5天，10天，20天的交易量的均值的比值
# 这里输入的数据为sAdBpC格式
def construct_daily_volume_moving_ratios(market_data_df):
    
    # 整合日交易量数据
    daily_volume_series = market_data_df.groupby(market_data_df.index.map(lambda x: x.split("p")[0]), sort = False).volume.sum()

    # 记录计算结果
    result_df = pd.DataFrame()
    
    # 开始计算
    result_df["daily_volume_moving_ratio_5d"] = daily_volume_series / daily_volume_series.rolling(5).mean()
    result_df["daily_volume_moving_ratio_10d"] = daily_volume_series / daily_volume_series.rolling(10).mean()
    result_df["daily_volume_moving_ratio_20d"] = daily_volume_series / daily_volume_series.rolling(20).mean()

    # 返回结果
    return result_df

# Feature构建函数：开盘后2个时间点 & 收盘前2个时间点的交易量和过去5天同时间点的交易量的均值的比值
# 这里输入的数据为sAdBpC格式
def construct_open_and_close_volume_moving_ratios(market_data_df):
    
    # 整合早盘和尾盘交易量数据
    morning_session_selected_data = market_data_df[market_data_df.index.map(lambda x: True if x.split("p")[1] in ["1", "2"] else False)]
    tail_session_selected_data = market_data_df[market_data_df.index.map(lambda x: True if x.split("p")[1] in ["49", "50"] else False)]
    morning_session_volume_series = morning_session_selected_data.groupby(morning_session_selected_data.index.map(lambda x: x.split("p")[0]), sort = False).volume.sum()
    tail_session_volume_series = tail_session_selected_data.groupby(tail_session_selected_data.index.map(lambda x: x.split("p")[0]), sort = False).volume.sum()

    # 记录计算结果
    result_df = pd.DataFrame()
    
    # 开始计算
    result_df["morning_session_volume_moving_ratio_5d"] = morning_session_volume_series / morning_session_volume_series.rolling(5).mean()
    result_df["tail_session_volume_moving_ratio_5d"] = tail_session_volume_series / tail_session_volume_series.rolling(5).mean()

    # 返回结果
    return result_df
    
# Feature构建函数：总体换手率，以及开盘后2个时间点 & 收盘前2个时间点的交易量的换手率
# 这里输入的market_data数据为sAdBpC格式
def construct_period_and_overall_turnover_ratios(market_data_df, fundamental_data_df):

    # 记录计算结果，并同时获取多种fundamental数据
    result_df = pd.DataFrame()
    result_df["turnoverRatio"] = fundamental_data_df["turnoverRatio"]
    result_df["transactionAmount"] = fundamental_data_df["transactionAmount"]
    result_df["pe_ttm"] = fundamental_data_df["pe_ttm"]
    result_df["pb"] = fundamental_data_df["pb"]
    result_df["ps"] = fundamental_data_df["ps"]
    result_df["pcf"] = fundamental_data_df["pcf"]

    # 整合早盘和尾盘交易量数据
    morning_session_selected_data = market_data_df[market_data_df.index.map(lambda x: True if x.split("p")[1] in ["1", "2"] else False)]
    tail_session_selected_data = market_data_df[market_data_df.index.map(lambda x: True if x.split("p")[1] in ["49", "50"] else False)]
    morning_session_volume_series = morning_session_selected_data.groupby(morning_session_selected_data.index.map(lambda x: x.split("p")[0]), sort = False).volume.sum()
    tail_session_volume_series = tail_session_selected_data.groupby(tail_session_selected_data.index.map(lambda x: x.split("p")[0]), sort = False).volume.sum()

    # 整合日交易量数据
    daily_volume_series = market_data_df.groupby(market_data_df.index.map(lambda x: x.split("p")[0]), sort = False).volume.sum()

    # 由于fundamental数据中未提供总股本数据，这里使用早盘和尾盘各自的交易量数据和日交易量数据做比值，从而生成早盘和尾盘的换手率
    result_df["morning_session_turnoverRatio"] = fundamental_data_df["turnoverRatio"] * morning_session_volume_series / daily_volume_series
    result_df["tail_session_turnoverRatio"] = fundamental_data_df["turnoverRatio"] * tail_session_volume_series / daily_volume_series
    
    # 返回结果
    return result_df

# 加工选择的Ta-Lib相关Features
# 这里输入的market_data数据为sAdBpC格式
def construct_talib_features(market_data_df):
    
    # 将数据从sAdBpC格式整合成sAdB形式，并筛选出关键的列
    open_p = market_data_df.groupby(market_data_df.index.map(lambda x: x.split("p")[0]), sort = False).open.head(1)
    open_p.index = open_p.index.map(lambda x: x.split("p")[0])
    close_p = market_data_df.groupby(market_data_df.index.map(lambda x: x.split("p")[0]), sort = False).close.tail(1)
    close_p.index = close_p.index.map(lambda x: x.split("p")[0])
    high_p = market_data_df.groupby(market_data_df.index.map(lambda x: x.split("p")[0]), sort = False).high.max()
    low_p = market_data_df.groupby(market_data_df.index.map(lambda x: x.split("p")[0]), sort = False).low.min()
    
    # 构建感兴趣的features
    feature_df = pd.DataFrame([])
    feature_df["SAR"] = ta.SAR(high_p, low_p, acceleration = 0, maximum = 0)
    feature_df["SAREXT"] = ta.SAREXT(high_p, low_p, startvalue = 0, offsetonreverse = 0, accelerationinitlong = 0, accelerationlong = 0, accelerationmaxlong = 0, accelerationinitshort = 0, accelerationshort = 0, accelerationmaxshort = 0)
    feature_df["RSI"] = ta.RSI(close_p, timeperiod = 14) - 50  # 由于该feature的范围为0~100，50为其均值，故将其进行demean处理
    feature_df["HT_DCPERIOD"] = ta.HT_DCPERIOD(close_p)
    feature_df["HT_PHASOR_inphase"], feature_df["HT_PHASOR_quadrature"] = ta.HT_PHASOR(close_p)
    feature_df["HT_SINE_sine"], feature_df["HT_SINE_leadsine"] = ta.HT_SINE(close_p)

    # 返回结果
    return feature_df

# 将全部构建feature的代码结合起来
# 这里输入的market_data数据为sAdBpC格式
def construct_all_features(market_data_df, fundamental_data_df):
    
    # 记录全部feature数据的list
    all_feature_list = []
    
    # 调用每个函数
    ctc_returns = construct_ctc_returns(market_data_df)
    daily_volume_moving_ratios = construct_daily_volume_moving_ratios(market_data_df)
    morning_and_tail_volume_moving_ratios = construct_open_and_close_volume_moving_ratios(market_data_df)
    period_and_overall_turnover_ratios = construct_period_and_overall_turnover_ratios(market_data_df, fundamental_data_df)
    all_feature_list.append(ctc_returns)
    all_feature_list.append(daily_volume_moving_ratios)
    all_feature_list.append(morning_and_tail_volume_moving_ratios)
    all_feature_list.append(period_and_overall_turnover_ratios)
    
    # 调用选择的ta-lib的Features
    talib_features = construct_talib_features(market_data_df)
    all_feature_list.append(talib_features)

    # 将这些数据结合起来
    all_features = pd.concat(all_feature_list, axis = 1)

    # 返回结果
    return all_features

# Y构建函数：获取当前数据下可获得的最新的y数据列
# 这里输入的数据为sAdBpC格式
def construct_Y(market_data_df):
    
    # 选出正确时间点的数据
    selected_data_df = market_data_df[market_data_df.index.map(lambda x: True if x.split("p")[1] == "50" else False)]
    selected_data_df.index = selected_data_df.index.map(lambda x: x.split("p")[0])
    
    # 开始计算
    y_to_predict_series = selected_data_df.close.pct_change(2).shift(-2)
    y_to_predict_series.name = "y_to_predict"

    # 返回结果
    return y_to_predict_series

# Process the train datasets & Train Models

In [3]:
# 加工模型训练数据
def prepare_train_dataset(market_data_dict, fundamental_data_dict):
    
    # 对每一个资产进行处理
    all_y_and_X_dict = {}
    for each_investment in market_data_dict:
        
        # 获取其各项数据，并构建y和X
        market_data_df = market_data_dict[each_investment]
        fundamental_data_df = fundamental_data_dict[each_investment]
        all_features = construct_all_features(market_data_df, fundamental_data_df)
        y_to_predict = construct_Y(market_data_df)
        
        # 将y和X结合起来，并根据train_length来筛选长度
        y_and_X_df = pd.concat([y_to_predict, all_features], axis = 1)
        y_and_X_df = y_and_X_df
        
        # 记录结果
        all_y_and_X_dict[each_investment] = y_and_X_df
    
    # 返回结果
    return all_y_and_X_dict

In [4]:
# Get the train datasets
df_train_market = pd.read_csv(TRAIN_MARKET_PATH)
df_train_market = df_train_market.set_index("date_time")
df_train_fundamental = pd.read_csv(TRAIN_FUNADMENTAL_PATH)
df_train_fundamental = df_train_fundamental.set_index("date_time")
market_data_dict = {}
fundamental_data_dict = {}
for i in range(54):
    stock_index = "s" + str(i)
    market_data_dict[stock_index] = df_train_market[df_train_market.index.map(lambda x: True if x.split("d")[0] == stock_index else False)]
    fundamental_data_dict[stock_index] = df_train_fundamental[df_train_fundamental.index.map(lambda x: True if x.split("d")[0] == stock_index else False)]

def correlation(a, train_data):
    b = train_data.get_label()
    a = np.ravel(a)
    b = np.ravel(b)
    len_data = len(a)
    mean_a = np.sum(a) / len_data
    mean_b = np.sum(b) / len_data
    var_a = np.sum(np.square(a - mean_a)) / len_data
    var_b = np.sum(np.square(b - mean_b)) / len_data
    cov = np.sum((a * b))/len_data - mean_a*mean_b
    corr = cov / np.sqrt(var_a * var_b)
    return "corr", corr, True
    
def train_model(train_time_length):
    
    # 准备训练初始模型所需的数据
    initial_all_y_and_X_dict = prepare_train_dataset(market_data_dict, fundamental_data_dict)

    # 处理每一个股票数据
    initial_train_df_component_list = []
    extreme_value_dict = {}
    for each_investment in initial_all_y_and_X_dict:
        temp_df = copy.deepcopy(initial_all_y_and_X_dict[each_investment])
        temp_df["investment_code"] = each_investment
        temp_df["date"] = temp_df.index.map(lambda x: int(x.split("d")[1]))
        temp_df = temp_df.set_index(["investment_code", "date"])
        temp_df = temp_df.iloc[-train_time_length:]

        # 移除异常值
        temp_df = temp_df.replace(np.inf, np.nan)
        temp_df = temp_df.dropna()
        initial_train_df_component_list.append(temp_df)

        # 处理并记录feature的极端值
        for each_feature in temp_df.columns[1:]:
            if each_feature in ["close_price"]:
                continue
            lower_quantile_value, upper_quantile_value = temp_df[each_feature].quantile(0.005), temp_df[each_feature].quantile(0.995)
            temp_df[each_feature] = temp_df[each_feature].apply(lambda x: upper_quantile_value if x > upper_quantile_value else x)
            temp_df[each_feature] = temp_df[each_feature].apply(lambda x: lower_quantile_value if x < lower_quantile_value else x)
            if each_feature not in extreme_value_dict:
                extreme_value_dict[each_feature] = {}
            extreme_value_dict[each_feature][each_investment] = [lower_quantile_value, upper_quantile_value]

    # 将数据集结合起来
    initial_train_df = pd.concat(initial_train_df_component_list)

    # 移除y的极端值
    lower_quantile_value, upper_quantile_value = initial_train_df["y_to_predict"].quantile(0.0002), initial_train_df["y_to_predict"].quantile(0.9998)
    initial_train_df["y_to_predict"] = initial_train_df["y_to_predict"].apply(lambda x: np.nan if x < lower_quantile_value or x > upper_quantile_value else x)
    initial_train_df = initial_train_df.dropna()
    
    # Set the hyperparameters
    n_fold = 10
    group_gap = 31
    seed = 42
    params = {
        'objective': 'rmse',  
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'verbose': -1
    }

    # Store all models trained
    all_models = []

    # Create a KFold object
    gkf = TimeSeriesSplit(n_splits = n_fold, gap = group_gap)
    for fold, (trn_ind, val_ind) in enumerate(gkf.split(initial_train_df.index.values)):    

        # Train a model for each fold
        print(f'Training fold {fold + 1}')
        x_train, x_val = initial_train_df.iloc[:, 1:].iloc[trn_ind], initial_train_df.iloc[:, 1:].iloc[val_ind]
        y_train, y_val = initial_train_df.iloc[:, 0].iloc[trn_ind], initial_train_df.iloc[:, 0].iloc[val_ind]
        train_dataset = lgb.Dataset(x_train, y_train)
        val_dataset = lgb.Dataset(x_val, y_val)
        model = lgb.train(params = params,
                            train_set = train_dataset,
                            valid_sets = [train_dataset, val_dataset],
                            num_boost_round = 200,
                            early_stopping_rounds = 20,
                            verbose_eval = False,
                            feval = correlation)

        # Save the model
        all_models.append(model)
    
    # Return all models
    return all_models, extreme_value_dict

In [5]:
### Train the initial model
train_time_length = 800
all_models, extreme_value_dict = train_model(train_time_length)

Training fold 1


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training fold 2
Training fold 3
Training fold 4
Training fold 5
Training fold 6
Training fold 7
Training fold 8
Training fold 9
Training fold 10


# Make Predictions

In [6]:
# 加工模型预测数据
def prepare_test_dataset(market_data_dict, fundamental_data_dict):
    
    # 对每一个资产进行处理
    all_X_dict = {}
    for each_investment in market_data_dict:
        
        # 获取其各项数据，并构建X
        market_data_df = market_data_dict[each_investment]
        fundamental_data_df = fundamental_data_dict[each_investment]
        all_features = construct_all_features(market_data_df, fundamental_data_df)
        
        # 取出最后一行并记录结果
        all_X_dict[each_investment] = all_features.iloc[[-1], :]
    
    # 返回结果
    return all_X_dict

In [7]:
# Use the provided Python API to submit the predictions
retrain_model_count = 0
test_time_length = 50
while not env.is_end():
    fundamental_df, market_df = env.get_current_market()
    prediction_list = []
    print(fundamental_df["date_time"].unique())
    
    # 将获取的数据结合到先前的dictionary中，不断更新数据
    market_df = market_df.set_index("date_time")
    fundamental_df = fundamental_df.set_index("date_time")
    for each_investment in market_data_dict:
        market_data_dict[each_investment] = market_data_dict[each_investment].append(market_df[market_df.index.map(lambda x: True if x.split("d")[0] == each_investment else False)])
        fundamental_data_dict[each_investment] = fundamental_data_dict[each_investment].append(fundamental_df[fundamental_df.index.map(lambda x: True if x.split("d")[0] == each_investment else False)])
    
    # Judge if retrain needed
    retrain_model_count += 1
    if retrain_model_count == test_time_length:
        retrain_model_count = 0
        all_models, extreme_value_dict = train_model(train_time_length)
    
    # 生成测试集的X
    all_X_dict = prepare_test_dataset(market_data_dict, fundamental_data_dict)
    
    # 处理feature的极端值
    for each_feature in extreme_value_dict:
        for each_investment in extreme_value_dict[each_feature]:
            lower_quantile_value, upper_quantile_value = extreme_value_dict[each_feature][each_investment][0], extreme_value_dict[each_feature][each_investment][1]
            all_X_dict[each_investment][each_feature] = all_X_dict[each_investment][each_feature].apply(lambda x: upper_quantile_value if x > upper_quantile_value else x)
            all_X_dict[each_investment][each_feature] = all_X_dict[each_investment][each_feature].apply(lambda x: lower_quantile_value if x < lower_quantile_value else x)
    
    # 对于每一个investment进行处理
    for each_investment in all_X_dict:
        temp_value = 0
        selected_X = copy.deepcopy(all_X_dict[each_investment])
        
        # 处理数据格式
        selected_X = selected_X.reset_index(drop = True)
        
        # 进行模型预测
        if selected_X.dropna().shape[0] == 0:
            pass
        else:
            for each_model in all_models:
                temp_value += each_model.predict(selected_X)[0]
        prediction_list.append(temp_value)
        
    # Submit the prediction for this date
    env.input_prediction(pd.Series(prediction_list))

['s0d1001' 's1d1001' 's2d1001' 's3d1001' 's4d1001' 's5d1001' 's6d1001'
 's7d1001' 's8d1001' 's9d1001' 's10d1001' 's11d1001' 's12d1001' 's13d1001'
 's14d1001' 's15d1001' 's16d1001' 's17d1001' 's18d1001' 's19d1001'
 's20d1001' 's21d1001' 's22d1001' 's23d1001' 's24d1001' 's25d1001'
 's26d1001' 's27d1001' 's28d1001' 's29d1001' 's30d1001' 's31d1001'
 's32d1001' 's33d1001' 's34d1001' 's35d1001' 's36d1001' 's37d1001'
 's38d1001' 's39d1001' 's40d1001' 's41d1001' 's42d1001' 's43d1001'
 's44d1001' 's45d1001' 's46d1001' 's47d1001' 's48d1001' 's49d1001'
 's50d1001' 's51d1001' 's52d1001' 's53d1001']
['s0d1002' 's1d1002' 's2d1002' 's3d1002' 's4d1002' 's5d1002' 's6d1002'
 's7d1002' 's8d1002' 's9d1002' 's10d1002' 's11d1002' 's12d1002' 's13d1002'
 's14d1002' 's15d1002' 's16d1002' 's17d1002' 's18d1002' 's19d1002'
 's20d1002' 's21d1002' 's22d1002' 's23d1002' 's24d1002' 's25d1002'
 's26d1002' 's27d1002' 's28d1002' 's29d1002' 's30d1002' 's31d1002'
 's32d1002' 's33d1002' 's34d1002' 's35d1002' 's36d1002' 's3